In [1]:
import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time
import pickle,h5py,json

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap,TSNE
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from scipy import fftpack
from scipy.signal import resample,correlate
from xgboost import XGBClassifier,XGBRegressor

import gym

import pandas_datareader as pdr
import urllib
from bs4 import BeautifulSoup
import networkx as nx
from lmfit import Model

import iexfinance as iex
from iexfinance.stocks import get_historical_data

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [28]:
tf.reset_default_graph()

x_0 = tf.placeholder(dtype=tf.float32,shape=[None,2],name='input0')

x = tf.layers.Dense(units=5,activation=tf.nn.relu)(x_0)
x = tf.layers.Dense(units=5,activation=tf.nn.relu)(x)
policy = tf.layers.Dense(units=1,activation=tf.nn.tanh)(x)

log_y = tf.log(policy)

#optimizer = tf.train.AdamOptimizer(0.1)
update = tf.gradients(log_y,tf.trainable_variables())

#update_and_vars = list(zip(update, tf.trainable_variables()))
#train_op = optimizer.apply_gradients(update_and_vars)

In [29]:
def play_episode(sess,env,N_max=500,verbose=False):

    state = env.reset()

    S = []
    A = []
    R = []
    
    timesteps = 0
    for _ in range(N_max):

        if verbose: env.render()
        timesteps += 1
        action = sess.run(policy,feed_dict={
            x_0:state.reshape(-1,state.shape[0])
        })
        state_next,reward,done,info = env.step(action)

        S.append(state)
        A.append(action)
        R.append(reward)

        if done: break
        state = state_next

    return S,A,R,timesteps

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

env = gym.make('MountainCarContinuous-v0')
S,A,R,n = play_episode(sess,env,verbose=True)
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [17]:
ud = sess.run(update,feed_dict={
    x_0:S[0].reshape(-1,2)
})

In [32]:
R

[-0.004672479745675507,
 -0.004675954586891984,
 -0.00467121203701959,
 -0.0046582903326714,
 -0.004637317176617728,
 -0.004608505890000303,
 -0.004572146298365554,
 -0.0045286083428663915,
 -0.004478324640050802,
 -0.004421803609948172,
 -0.004359602867749857,
 -0.004292332738177951,
 -0.004220644452074396,
 -0.004145217902078358,
 -0.004066762395485402,
 -0.003985998149860693,
 -0.003903648826070616,
 -0.0038204404472434784,
 -0.0037370798434146705,
 -0.003654260453996217,
 -0.003572649599972189,
 -0.003492883794158375,
 -0.0034155638521451205,
 -0.003341249324706386,
 -0.0032704593290200147,
 -0.003203669161045775,
 -0.003141309293575034,
 -0.0030837628575697273,
 -0.003031375084470156,
 -0.00298443991716566,
 -0.0029432170422280858,
 -0.002907924808597784,
 -0.0028787464152586263,
 -0.002855823993341833,
 -0.002839273417345467,
 -0.002829176671822609,
 -0.0028255801309456045,
 -0.0028285115127229202,
 -0.002837959883649788,
 -0.0028538878453782113,
 -0.002876229323921198,
 -0.00290